# 資料抓取與儲存成 DB

In [3]:
import time
import sqlite3
import json
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datetime import datetime

# ===== 基本設定 =====
BASE = "https://sea.cc.ntpu.edu.tw/pls/dev_stud/"
LIST_URL = urljoin(BASE, "course_query_all.queryByAllConditions")

# 查詢條件（學年度、學期）
BASE_PAYLOAD = {
    "qEdu": "",
    "qCollege": "",
    "qdept": "",
    "qYear": "114",
    "qTerm": "1",
    "qGrade": "",
    "qClass": "",
    "cour": "",
    "teach": "",
    "qMemo": "",
    "week": "",
    "seq1": "A",
    "seq2": "M",
}

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Content-Type": "application/x-www-form-urlencoded",
    "Origin": "https://sea.cc.ntpu.edu.tw",
    "Referer": urljoin(BASE, "course_query_all.CHI_query_Common"),
}

# 學制對應表
EDU_TYPES = {
    "U": "學士班",
    "N": "進修學士班",
    "M": "碩博士班",
    "P": "碩士在職專班",
}


# ===== 輔助函數 =====
def td_text(td, sep="|"):
    return td.get_text(separator=sep, strip=True).replace("\xa0", "").strip()


def split_name_and_note(raw_text):
    key = "備註："
    if key in raw_text:
        name, note = raw_text.split(key, 1)
        return name.strip(), note.strip()
    return raw_text.strip(), ""


def parse_name_cell(td):
    links = td.find_all("a", href=True)
    syllabus_url, limit_url = None, None
    for a in links:
        href = a["href"]
        if "queryGuide2" in href:
            syllabus_url = urljoin(BASE, href)
        if "bylimit_checkshow" in href:
            limit_url = urljoin(BASE, href)

    raw = td.get_text(separator="\n", strip=True)
    name_text, note_text = split_name_and_note(raw)
    name_lines = [l for l in name_text.splitlines() if l.strip()]
    course_name = " / ".join(name_lines) if name_lines else ""

    return course_name, note_text, syllabus_url, limit_url


def fetch_limit_table(sess, url):
    """抓取限制頁 → 回傳 dict"""
    try:
        res = sess.get(url, headers=HEADERS, timeout=30)
        res.encoding = "big5"
        soup = BeautifulSoup(res.text, "html.parser")
        table = soup.find("table")
        if not table:
            return {}
        restrictions = {}
        for tr in table.find_all("tr"):
            tds = tr.find_all("td")
            if len(tds) == 2:
                key = tds[0].get_text(strip=True)
                if key == "群組":  # 忽略群組
                    continue
                val = tds[1].get_text(strip=True)
                restrictions[key] = val
        return restrictions
    except Exception as e:
        print(f"⚠️ 抓限制失敗 {url}: {e}")
        return {}


def ensure_db():
    conn = sqlite3.connect("ntpu_courses.db")
    cur = conn.cursor()
    cur.executescript("""
    CREATE TABLE IF NOT EXISTS courses (
        yearterm TEXT,
        serial TEXT,
        dept TEXT,
        grade TEXT,
        required TEXT,
        name TEXT,
        note TEXT,
        teacher TEXT,
        category TEXT,
        credit INTEGER,
        hours INTEGER,
        language TEXT,
        schedule TEXT,
        addable TEXT,
        add_limit INTEGER,
        total_limit INTEGER,
        enrolled INTEGER,
        syllabus_url TEXT,
        limit_url TEXT,
        limits_json TEXT,
        edu_type TEXT,
        crawl_time TEXT,
        PRIMARY KEY (yearterm, serial, edu_type)
    );
    """)
    conn.commit()
    return conn


# ===== 主流程 =====
def main():
    sess = requests.Session()
    conn = ensure_db()
    cur = conn.cursor()

    total = 0
    crawl_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    for code, edu_name in EDU_TYPES.items():
        print(f"\n===== 抓取 {edu_name} 課程中 =====")

        payload = BASE_PAYLOAD.copy()
        payload["qEdu"] = code

        try:
            res = sess.post(LIST_URL, headers=HEADERS, data=payload, timeout=60)
            res.encoding = "big5"
        except Exception as e:
            print(f"⚠️ 請求失敗：{edu_name} {e}")
            continue

        soup = BeautifulSoup(res.text, "html.parser")
        tables = soup.find_all("table")
        if not tables:
            print(f"⚠️ {edu_name} 找不到課程表格")
            continue

        # 找到課程最多的表格
        table = max(tables, key=lambda t: len(t.find_all("tr")))
        rows = table.find_all("tr")[1:]

        count = 0
        with_limits = 0

        for tr in rows:
            tds = tr.find_all("td")
            if len(tds) < 18:
                continue

            year = td_text(tds[1])
            term = td_text(tds[2])
            serial = td_text(tds[3])
            yearterm = year + term

            dept = td_text(tds[4])
            grade = td_text(tds[5], sep="|")
            required = td_text(tds[6], sep="|")
            name, note, syllabus_url, limit_url = parse_name_cell(tds[7])
            teacher = td_text(tds[8], sep="|")
            category = td_text(tds[9])
            credit = td_text(tds[10])
            hours = td_text(tds[11])
            language = td_text(tds[12])
            schedule = td_text(tds[13], sep="; ")
            addable = td_text(tds[14])
            add_limit = td_text(tds[15])
            total_limit = td_text(tds[16])
            enrolled = td_text(tds[17])

            limits_dict = {}
            if limit_url:
                limits_dict = fetch_limit_table(sess, limit_url)
                if limits_dict:
                    with_limits += 1
                time.sleep(0.5)  # 禮貌延遲

            cur.execute("""
            INSERT OR REPLACE INTO courses (
                yearterm, serial, dept, grade, required, name, note, teacher,
                category, credit, hours, language, schedule, addable, add_limit,
                total_limit, enrolled, syllabus_url, limit_url, limits_json,
                edu_type, crawl_time
            ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
            """, (
                yearterm, serial, dept, grade, required, name, note, teacher,
                category, credit, hours, language, schedule, addable, add_limit,
                total_limit, enrolled, syllabus_url, limit_url,
                json.dumps(limits_dict, ensure_ascii=False),
                edu_name, crawl_time
            ))

            count += 1

        conn.commit()
        total += count
        print(f"✅ 完成 {edu_name}：共 {count} 筆，其中 {with_limits} 門有課程限制。")
        time.sleep(2)  # 學制之間延遲，避免過快

    conn.close()
    print(f"\n🎉 所有學制完成，共匯入 {total} 筆課程。")
    print(f"📅 抓取時間：{crawl_time}")


if __name__ == "__main__":
    main()


===== 抓取 學士班 課程中 =====
✅ 完成 學士班：共 1370 筆，其中 581 門有課程限制。

===== 抓取 進修學士班 課程中 =====
✅ 完成 進修學士班：共 394 筆，其中 178 門有課程限制。

===== 抓取 碩博士班 課程中 =====
✅ 完成 碩博士班：共 726 筆，其中 154 門有課程限制。

===== 抓取 碩士在職專班 課程中 =====
✅ 完成 碩士在職專班：共 104 筆，其中 33 門有課程限制。

🎉 所有學制完成，共匯入 2594 筆課程。
📅 抓取時間：2025-11-05 00:38:39
